<a href="https://colab.research.google.com/github/josepmatias/ONE/blob/main/auto_campanhas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pip install pyxlsb

In [3]:
#nomear os ficheiros

#activacoes="auto_listagemactivacoes.xlsb"
#auto="03.2024 - Auto Upsell_Prospect_Rec_Cancelados_formulas.xlsb"
modelo_upfront="modelo upfront.xlsb"
activacoes="ONE_Ativações_Serviços_Marcações_Migrações_mar24.xlsb"
cadastro1="Cadastro.xlsb"

In [4]:
#carregar ficheiro de activações e auto

from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()
uploaded3=files.upload()

Saving ONE_Ativações_Serviços_Marcações_Migrações_mar24.xlsb to ONE_Ativações_Serviços_Marcações_Migrações_mar24.xlsb


Saving modelo upfront.xlsb to modelo upfront.xlsb


Saving Cadastro.xlsb to Cadastro.xlsb


In [ ]:
#from google.colab import files
#uploaded3 = files.upload()

Saving modelo upfront.xlsb to modelo upfront.xlsb


In [5]:
#ler ficheiro db_upfront
#db_upfront usada para o valor upfront €, excepto quando análise daf é "DTH para Cabo" e "Pré-Pago"
db_upfront=pd.read_excel(modelo_upfront, sheet_name="modelo upfront", header=1).iloc[:, [2, 3, 4]]

db_upfront


,Sub-Área de Negócio,Upfront,Pts GCO
0,ALARME,25.0,depende da campanha
1,CABO,10.0,depende da campanha
2,Desporto,2.0,0
3,DTH,10.0,depende da campanha
4,DTH para Cabo,5.0,1.25
5,IOT,3.0,depende da campanha
6,Marcações,1.5,depende da campanha
7,Migrações Stand Alone,2.5,depende da campanha
8,NAVEGAÇÃO SEGURA,0.5,0
9,NET,10.0,depende da campanha


In [6]:
#ler ficheiro db_pontos_gco
#db_pontos_gco é usada para fixar pontos (coluna pts GCO) excepto quando a análise daf é "Comissionada Navegação Segura", "Comissionado Seguro" \
#"Comissionado Router 6.0", ""Comissionado Power wi-fi""
db_pontos_gco=pd.read_excel(modelo_upfront, sheet_name="modelo upfront", header=1).iloc[:, 8:10]
db_pontos_gco.head()

,Campanha,pts GCO
0,lm_churn,0.75
1,lm_churn_ac,0.75
2,prospect_alarmes,3.00
3,prospect_blitz_blm,2.00
4,prospect_blitz_wow,2.00


In [7]:
#ler ficheiro da listagem de activações
auto_db=pd.read_excel(activacoes, sheet_name="Detalhe")
print(auto_db.shape)
auto_db.head()

(8779, 47)


,Mês de Fecho,Order Item,Chave,SFID,Comissionamento,Sub-Área de Negócio,Service Account,Customer Account (SA),CA&UA,Alojamento Id,...,Notas DS,Upfront €,pts GCO,GCO €,Total €,Final,Analise DAF,Pagar,Conclusão SDD,Análise SDD
0,45352,1-KLM2AVR,40168940583780086295,L.G01.UP1.043,ONE,CABO,S965395918,C145652802,C1456528027901704365,8996880.0,...,Uso da campanha final ou wise dará pts de GCO ...,10.0,3.00,2.0,12.0,ok,Comissionado,Sim,NaN,NaN
1,45352,1-KNXHCJ1,40168940583780047659,L.G01.UP1.011,ONE,CABO,S825253392,C172136602,C1721366021255016819,2122847.0,...,NaN,10.0,3.00,2.0,12.0,ok,Comissionado,Sim,NaN,NaN
2,45352,1-KMQRCQB,20544240583580122712,B.C01.UP1.287,ONE,CABO,S846897051,C173519001,C1735190017914773340,2244259.0,...,NaN,10.0,3.00,2.0,12.0,ok,Comissionado,Sim,NaN,NaN
3,45352,1-KLG8LET,40168940583770080648,L.G01.UP1.627,ONE,CABO,S965388611,C174106801,C17410680110012563678,4146708.0,...,NaN,10.0,2.00,-7.5,2.5,ok,Comissionado Blitz BLM,Sim,NaN,NaN
4,45352,1-KOOQNHS,40168941003680079762,L.G01.RMC.006,ONE,CABO,S848815192,C225385001,C22538500110016569851,1565886.0,...,NaN,5.0,1.25,1.0,6.0,ok,Comissionado DTH para Cabo,Sim,NaN,NaN


In [8]:
#processo de cálculo de upfront para operador
def upfront (db, db2, str1="DTH para Cabo", str2="Pré-Pago"):

  db_trad=db.copy()
  for cont in range(3):
    if cont==0:
      indice=db.loc[(db["Pagar"]=="Sim") & (db["Analise DAF"].str.find(str1)==-1) & (db["Analise DAF"].str.find(str2)==-1)].index
      area_negocio=db.loc[indice, "Sub-Área de Negócio"].unique()
      dici={}
      for i in area_negocio:
        dici[i]=db2.loc[db2.iloc[:, 0]==i, "Upfront"].tolist()[0]
      db_trad.loc[indice, "upfront"]=db_trad.loc[indice, "Sub-Área de Negócio"].map(dici)
    elif cont==1:
      indice=db.loc[(db["Pagar"]=="Sim") & (db["Analise DAF"].str.find(str1)!=-1)].index
      area_negocio2=db.loc[indice, "Sub-Área de Negócio"].apply(lambda i: i+str1)
      for i in area_negocio2.unique():
        dici[i]=db2.loc[db2.iloc[:, 0]==i, "Upfront"].tolist()[0]
      db_trad.loc[indice, "upfront"]=db_trad.loc[indice, "Sub-Área de Negócio"].apply(lambda i: i+str1).map(dici)
    elif cont==2:
      indice=db.loc[(db["Pagar"]=="Sim") & (db["Analise DAF"].str.find(str2)!=-1)].index
      area_negocio3=db.loc[indice, "Sub-Área de Negócio"].apply(lambda i: i+str2)
      for i in area_negocio3.unique():
        dici[i]=db2.loc[db2.iloc[:, 0]==i, "Upfront"].tolist()[0]
      db_trad.loc[indice, "upfront"]=db_trad.loc[indice, "Sub-Área de Negócio"].apply(lambda i: i+str2).map(dici)
  return db_trad["upfront"]

db_up=pd.concat([auto_db, upfront(auto_db, db_upfront)], axis=1)

In [24]:
#"Comissionada Navegação Segura", "Comissionado Seguro", "Comissionado Router 6.0", "Comissionado Power wi-fi" -> análise daf
no_points=["Comissionada Navegação Segura", "Comissionado Seguro", "Comissionado Router 6.0", "Comissionado Power wi-fi"]
def pontos_op(db, db2, lista=no_points, col="Campanha wise"):
  db_pontos=db.copy()

  for j in range(2):
    if j==0:
      indice=db.loc[(db["Pagar"]=="Sim") & ~(db["Analise DAF"].isin(lista)), col].index
      #retirar string "Info comercial: " e lidar com caracter estranho
      #campanhas=db.loc[(db["Pagar"]=="Sim") & ~(db["Analise DAF"].isin(lista)), col].str.replace("Info comercial: ", "").unique().tolist()

      campanhas=db.loc[(db["Pagar"]=="Sim") & ~(db["Analise DAF"].isin(lista)), col].unique().tolist()

      #encontrar os pts para a campanha
      dici={}
      for i in campanhas:
        if i.find("Info comercial: ")!=-1:
          #lidar com companhas que incluem "Info comercial "
          str3=i.replace("Info comercial: ", "")
          dici[i]=db2.loc[db2.iloc[:, 0]==str3, "pts GCO"].tolist()[0]
        else:
          dici[i]=db2.loc[db2.iloc[:, 0]==i, "pts GCO"].tolist()[0]

      #criar coluna pontos a partir da campanha wise
      db_pontos.loc[indice, "pontos"]=db.loc[indice, col].map(dici)

  #colocar pontos da lista a zero
    else:
      indice=db.loc[(db["Pagar"]=="Sim") & (db["Analise DAF"].isin(lista)), col].index

      db_pontos.loc[indice, "pontos"]=0

  return db_pontos["pontos"]

db_up2=pd.concat([db_up, pontos_op(db_up, db_pontos_gco)], axis=1)

In [19]:
#criar objectivos, atingimento e gco para os operadores

#somar pontos de cada operador
db_gco=db_up2[["user", "Carga Horária", "pontos"]].groupby(["user", "Carga Horária"]) ["pontos"].sum().reset_index()

#definição objectivo de cada operador: 4h - 70; 8h - 125
db_gco["objectivo"]=[70 if i==4 else 125 for i in db_gco["Carga Horária"].tolist()]

#definição do atingimento gco para cada operador
db_gco["atingimento_gco"]=db_gco["pontos"]/db_gco["objectivo"]

db_gco.head()

,user,Carga Horária,pontos,objectivo,atingimento_gco
0,AJALMEIDA,8,150.0,125,1.200000
1,CMMACIEIRA,4,106.0,70,1.514286
2,CPSARDON,8,172.0,125,1.376000
3,CRREIS,8,206.0,125,1.648000
4,LSROCHA,8,186.0,125,1.488000


In [25]:
#definir na datatable "db_up2" o atingimento gco para cada operador

db_up2["atingimento_gco"]=db_up2.merge(db_gco[["user", "atingimento_gco"]], how="left", on="user") ["atingimento_gco"]

db_up2["atingimento_gco"]

0       1.544000
1       1.712000
2       1.504000
3       0.816000
4       1.630000
          ...   
8774    1.007143
8775    1.007143
8776    1.007143
8777    1.007143
8778    1.007143
Name: atingimento_gco, Length: 8779, dtype: float64

In [26]:
#cálculo do valor upfront+gco
def gco_euro(val1, val2):
  #valor_gco=[]
  if val2>=1.5:
    valor_gco=val1*0.2
  elif val2>=1.3 and val2<1.5:
    valor_gco=val1*0.1
  elif val2>=1.1 and val2<1.3:
    valor_gco=val1*0.05
  elif val2>=1.0 and val2<1.1:
    valor_gco=0
  elif val2>=0.9 and val2<1.0:
    valor_gco=val1*-1*0.5
  elif val2>=0.8 and val2<0.9:
    valor_gco=val1*-1*0.75
  else:
    valor_gco=-val1
  return valor_gco

db_up2["gco_euro"]=db_up2[["upfront", "atingimento_gco"]].apply(lambda i: gco_euro(i["upfront"], i["atingimento_gco"]), axis=1)
db_up2["valor_final"]=db_up2["upfront"]+db_up2["gco_euro"]

In [27]:
db_up2.shape

(8779, 52)

In [13]:
#carregar cadastro para associar
cadastro=pd.read_excel(cadastro1, sheet_name=0)
cadastro.head()

,SFID,Cav outbound,Nome,Nº Colaborador,User Aplicações,Userde Login/consolas,CAV Inbound,CAV Outbound,CAV Alternativo,Data de Inicio atendimento,...,Rede,Linha,Data Inicio de Campanha,Data Fim de Campanha,Campanha Final,Faltas,Férias,Elegível GCO GE,Parceiro,SFID´S
0,X.V03.IN1.109,80043208,Patrícia Isabel Teixeira Moreira,80043208,U80043208,iqn,NaN,80043208,NaN,41472,...,ONE Upsell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Randstad,X.V03.IN1.109
1,L.G01.UP1.322,80125104,Ana Sofia Rosinha Ferreira,80125104,anasofferreira,asr,NaN,80125104,NaN,2019-02-18,...,ONE Prospect,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Heading,L.G01.UP1.322
2,L.G01.UP2.047,70048974,Telma Isabela Machado Coelho,80048974,U80048974,tic,NaN,70048974,NaN,41702,...,ONE Upsell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Heading,L.G01.UP2.047
3,L.G01.UP1.259,80047459,Ermelinda Catarina Marques Pereira Teles,80047459,U80047459,ecm,NaN,80047459,NaN,41610,...,ONE Upsell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Heading,L.G01.UP1.259
4,B.C01.UP1.471,70044985,Filipa Alexandra Lemos Costa,70044985,U80044985,fac,NaN,70044985,NaN,44627,...,ONE Prospect,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Randstad,B.C01.UP1.471


In [31]:
cadastro2=cadastro[["User Aplicações"]].drop_duplicates()
print(cadastro2.shape)
cadastro2.head()

(150, 1)


,User Aplicações
0,U80043208
1,anasofferreira
2,U80048974
3,U80047459
4,U80044985


In [33]:
db_up2.columns

db_up3=db_up2.merge(cadastro2, how="left", left_on="user", right_on="User Aplicações")

print(db_up3.shape)

(8779, 53)


In [ ]:
#teste para ver se o gco calculado para operador está ok
lista1=db_up2.groupby("user")["atingimento_gco"].sum()
lista2=auto_db.groupby("user") ["GCO"].sum()

for i, j in zip(lista1, lista2):
  if i!=j:
    print("acabou")
    break

